First, establish an ssh tunnel [with your PNI password] to jtb3-dev.princeton.edu to make the database available to you:

    ssh username@jtb3-dev.princeton.edu -L 3306:127.0.0.1:3306 -N

jtb3-dev is the database server on which all behavior and mouse data is stored. Second, make sure you have installed datajoint >= '0.12.dev3'
[ pip install datajoint==0.12.dev3 ]

In [141]:
import pylab as pl
import datajoint as dj
import numpy as np
import os,sys
import secrets
assert dj.__version__>='0.12.dev3'
import pickle
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import pkg_resources
from shutil import copyfile

home_dir = os.environ['HOME']

In [142]:
dj.__version__

'0.12.dev7'

In [2]:
schema = dj.schema("ahoag_test_datatypes")

Please enter DataJoint username: ahoag
Please enter DataJoint password: ········
Connecting ahoag@localhost:3306


In [3]:
@schema
class User(dj.Lookup):
    definition = """
    # Users of the light sheet microscope
    username : varchar(20)      # user in the lab
    ---
    email       : varchar(50)
    """
    
@schema
class Experiment(dj.Manual):
    definition = """ # Experiments performed using the light sheet microscope
    experiment_id           :   smallint auto_increment    # allowed here are sql datatypes.
    ----
    cell_detection          :   boolean

    """  

In [5]:
insert_dict = {'cell_detection':True}
Experiment().insert1(insert_dict)
Experiment()

experiment_id allowed here are sql datatypes.,cell_detection
1,1


In [17]:
insert_dict = {'cell_detection':'0'}
Experiment().insert1(insert_dict)
Experiment()

experiment_id allowed here are sql datatypes.,cell_detection
1,1
2,1
3,2
4,-1
5,0
6,-100
7,0
8,10
9,-10
10,0


In [169]:
schema = dj.create_virtual_module("ahoag_lightsheet_demo","ahoag_lightsheet_demo")

In [170]:
samples_contents = schema.Sample()
exp_contents = schema.Experiment()
samples_contents

username user in the lab,experiment_name,sample_name,perfusion_date,expected_handoff_date,clearer netid of person doing the clearing,clearing_protocol,clearing_progress,antibody1,antibody2,notes_for_clearer,imager netid of person doing the imaging,imaging_progress,image_resolution,channel488_registration,channel555_registration,channel647_registration,channel790_registration,channel488_injection_detection,channel555_injection_detection,channel647_injection_detection,channel790_injection_detection,channel488_probe_detection,channel555_probe_detection,channel647_probe_detection,channel790_probe_detection,channel488_cell_detection,channel555_cell_detection,channel647_cell_detection,channel790_cell_detection,notes_for_imager,processor netid of person doing the processing,processing_progress,stitching_method,blend_type,atlas_name,tiling_overlap,intensity_correction
ahoag,asdf,test-1,None,None,not yet assigned,iDISCO abbreviated clearing,incomplete,,,,not yet assigned,incomplete,1.3x,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,asdf,test-2,None,None,not yet assigned,iDISCO abbreviated clearing,incomplete,,,,not yet assigned,incomplete,1.3x,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,t,t-1,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,ahoag,complete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,t,t-2,None,None,ahoag,iDISCO abbreviated clearing,complete,,,,ahoag,complete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-1,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-2,None,None,not yet assigned,uDISCO,incomplete,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-3,None,None,not yet assigned,iDISCO_EdU,incomplete,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test3,t-1,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1


In [171]:
exp_contents

username user in the lab,experiment_name,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,sample_prefix
ahoag,asdf,2019-11-14,15:10:46,asdf,ahoag@princeton.edu,asdf,mouse,2,test
ahoag,t,2019-11-14,10:51:42,braincogs,ahoag@princeton.edu,t,mouse,2,t
ahoag,test2,2019-11-14,15:13:55,wang,ahoag@princeton.edu,test,mouse,3,t
ahoag,test3,2019-11-14,15:20:09,wang,ahoag@princeton.edu,test3,mouse,1,t


In [180]:
(exp_contents * samples_contents).proj()

username user in the lab,experiment_name,sample_name
ahoag,asdf,test-1
ahoag,asdf,test-2
ahoag,t,t-1
ahoag,t,t-2
ahoag,test2,t-1
ahoag,test2,t-2
ahoag,test2,t-3
ahoag,test3,t-1


In [182]:
(exp_contents * samples_contents)

username user in the lab,experiment_name,sample_name,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,sample_prefix,perfusion_date,expected_handoff_date,clearer netid of person doing the clearing,clearing_protocol,clearing_progress,antibody1,antibody2,notes_for_clearer,imager netid of person doing the imaging,imaging_progress,image_resolution,channel488_registration,channel555_registration,channel647_registration,channel790_registration,channel488_injection_detection,channel555_injection_detection,channel647_injection_detection,channel790_injection_detection,channel488_probe_detection,channel555_probe_detection,channel647_probe_detection,channel790_probe_detection,channel488_cell_detection,channel555_cell_detection,channel647_cell_detection,channel790_cell_detection,notes_for_imager,processor netid of person doing the processing,processing_progress,stitching_method,blend_type,atlas_name,tiling_overlap,intensity_correction
ahoag,asdf,test-1,2019-11-14,15:10:46,asdf,ahoag@princeton.edu,asdf,mouse,2,test,None,None,not yet assigned,iDISCO abbreviated clearing,incomplete,,,,not yet assigned,incomplete,1.3x,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,asdf,test-2,2019-11-14,15:10:46,asdf,ahoag@princeton.edu,asdf,mouse,2,test,None,None,not yet assigned,iDISCO abbreviated clearing,incomplete,,,,not yet assigned,incomplete,1.3x,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,t,t-1,2019-11-14,10:51:42,braincogs,ahoag@princeton.edu,t,mouse,2,t,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,ahoag,complete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,t,t-2,2019-11-14,10:51:42,braincogs,ahoag@princeton.edu,t,mouse,2,t,None,None,ahoag,iDISCO abbreviated clearing,complete,,,,ahoag,complete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-1,2019-11-14,15:13:55,wang,ahoag@princeton.edu,test,mouse,3,t,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-2,2019-11-14,15:13:55,wang,ahoag@princeton.edu,test,mouse,3,t,None,None,not yet assigned,uDISCO,incomplete,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test2,t-3,2019-11-14,15:13:55,wang,ahoag@princeton.edu,test,mouse,3,t,None,None,not yet assigned,iDISCO_EdU,incomplete,,,,not yet assigned,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1
ahoag,test3,t-1,2019-11-14,15:20:09,wang,ahoag@princeton.edu,test3,mouse,1,t,None,None,ahoag,iDISCO abbreviated clearing,in progress,,,,ahoag,incomplete,1.3x,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,not yet assigned,incomplete,blending,sigmoidal,allen_2017,0.1,1


In [181]:
exp_contents.proj(datetime='TIMESTAMP(date_submitted,time_submitted)')

username user in the lab,experiment_name,datetime calculated attribute
ahoag,asdf,2019-11-14 15:10:46
ahoag,t,2019-11-14 10:51:42
ahoag,test2,2019-11-14 15:13:55
ahoag,test3,2019-11-14 15:20:09


In [146]:
schema = dj.create_virtual_module("ahoag_lightsheet_demo","ahoag_lightsheet_demo")

In [147]:
schema.Sample()

username user in the lab,experiment_name,sample_name,perfusion_date,expected_handoff_date,clearer netid of person doing the clearing,clearing_protocol,clearing_progress,antibody1,antibody2,notes_for_clearer,imager netid of person doing the imaging,imaging_progress,image_resolution,channel488_registration,channel555_registration,channel647_registration,channel790_registration,channel488_injection_detection,channel555_injection_detection,channel647_injection_detection,channel790_injection_detection,channel488_probe_detection,channel555_probe_detection,channel647_probe_detection,channel790_probe_detection,channel488_cell_detection,channel555_cell_detection,channel647_cell_detection,channel790_cell_detection,notes_for_imager,processor netid of person doing the processing,processing_progress,stitching_method,blend_type,atlas_name,tiling_overlap,intensity_correction


In [162]:
var='abc'
foo='bar'

In [86]:

schema.Sample().fetch?

Object `fetch` not found.


In [163]:
ng = (f"this is an {var} "
      f"implicitly joined {foo} string")
ng

'this is an abc implicitly joined bar string'

In [164]:
mystr = 'channel488_registration'
mystr[0:6]

'channe'

In [166]:
import os

In [ ]:
mkdir "test


In [93]:
sample_contents = schema.Sample() & 'sample_name ="test-1"'

In [96]:
sample_dict=sample_contents.fetch

In [103]:
Sample = schema.Sample()

In [136]:
contents = schema.Sample()

In [139]:
contents2=contents & 'sample_name="nothing"'

In [140]:
contents2.heading.attributes.keys()


dict_keys(['username', 'experiment_name', 'sample_name', 'perfusion_date', 'expected_handoff_date', 'clearer', 'clearing_protocol', 'clearing_progress', 'antibody1', 'antibody2', 'imager', 'imaging_progress', 'image_resolution', 'channel488_registration', 'channel555_registration', 'channel647_registration', 'channel790_registration', 'channel488_injection_detection', 'channel555_injection_detection', 'channel647_injection_detection', 'channel790_injection_detection', 'channel488_probe_detection', 'channel555_probe_detection', 'channel647_probe_detection', 'channel790_probe_detection', 'channel488_cell_detection', 'channel555_cell_detection', 'channel647_cell_detection', 'channel790_cell_detection', 'notes_for_clearer', 'notes_for_imager'])

In [135]:
Sample.heading.attributes.keys()

dict_keys(['username', 'experiment_name', 'sample_name', 'perfusion_date', 'expected_handoff_date', 'clearer', 'clearing_protocol', 'clearing_progress', 'antibody1', 'antibody2', 'imager', 'imaging_progress', 'image_resolution', 'channel488_registration', 'channel555_registration', 'channel647_registration', 'channel790_registration', 'channel488_injection_detection', 'channel555_injection_detection', 'channel647_injection_detection', 'channel790_injection_detection', 'channel488_probe_detection', 'channel555_probe_detection', 'channel647_probe_detection', 'channel790_probe_detection', 'channel488_cell_detection', 'channel555_cell_detection', 'channel647_cell_detection', 'channel790_cell_detection', 'notes'])

In [97]:
sample_dict

{'username': 'ahoag',
 'experiment_name': 'Test experiment',
 'sample_name': 'test-1',
 'perfusion_date': None,
 'expected_handoff_date': None,
 'clearer': 'ahoag',
 'clearing_protocol': 'iDISCO abbreviated clearing',
 'clearing_progress': 'in progress',
 'antibody1': '',
 'antibody2': '',
 'imager': 'ahoag',
 'imaging_progress': 'in progress',
 'image_resolution': '1.3x',
 'channel488_registration': 1,
 'channel555_registration': 0,
 'channel647_registration': 0,
 'channel790_registration': 0,
 'channel488_injection_detection': 0,
 'channel555_injection_detection': 1,
 'channel647_injection_detection': 0,
 'channel790_injection_detection': 0,
 'channel488_probe_detection': 0,
 'channel555_probe_detection': 0,
 'channel647_probe_detection': 0,
 'channel790_probe_detection': 0,
 'channel488_cell_detection': 0,
 'channel555_cell_detection': 0,
 'channel647_cell_detection': 0,
 'channel790_cell_detection': 0,
 'notes': ''}

In [101]:
l1=[0,0,1]
any(l1)

True

In [100]:
[key for key in sample_dict.keys() if key[0:10]=='channel488']

['channel488_registration',
 'channel488_injection_detection',
 'channel488_probe_detection',
 'channel488_cell_detection']

In [65]:
schema = dj.create_virtual_module("ahoag_microscope_demo","ahoag_microscope_demo")

/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/datajoint/connection.py:212: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [67]:
schema.Microscope()

microscope_name,center,room_number,optical_bay,loc_on_table,microscope_description


In [63]:
schema.Experiment

<bound method Table.describe of *username    *experiment_na date_submitted time_submitted labname       correspondence description    species     number_of_samp sample_prefix 
+----------+ +------------+ +------------+ +------------+ +-----------+ +------------+ +------------+ +---------+ +------------+ +------------+
ahoag        Test           2019-11-06     15:04:35       braincogs     ahoag@princeto t              mouse       5              t             
 (Total: 1)
>

In [42]:
user=schema.User()

In [44]:
user_contents = user & 'username="ahoag"'
user_contents

username user in the lab,princeton_email
ahoag,ahoag@princeton.edu


In [59]:
frame = pd.DataFrame(user_contents.fetch())
frame.username

0    ahoag
Name: username, dtype: object

In [37]:
insert_dict =  {'username':'ahoag','experiment_name':'Test','sample_name':'t-1','exp_notes':'notes'}
schema.Sample.IdiscoAbbreviatedClearing().insert1(insert_dict)

DuplicateError: ("Duplicate entry 'ahoag-Test-t-1' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [ ]:
schema.Sample.IdiscoAbbreviatedClearing().insert1()

In [11]:
user_contents=schema.User()

In [23]:
username='ahoag'
experiment_name='test3'

In [25]:
schema.Experiment()

username user in the lab,experiment_name,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,sample_prefix
ahoag,test,2019-11-04,14:28:07,wang,ahoag@princeton.edu,test,mouse,3,prefix
ahoag,test2,2019-11-04,14:47:31,wang,ahoag@princeton.edu,second test to try to get multiple samples to insert,mouse,2,sample
ahoag,test3,2019-11-04,14:48:28,wang,ahoag@princeton.edu,A third test,mouse,2,sample


In [26]:
schema.Experiment() & f'username="{username}"' & f'experiment_name="{experiment_name}"'

username user in the lab,experiment_name,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,sample_prefix
ahoag,test3,2019-11-04,14:48:28,wang,ahoag@princeton.edu,A third test,mouse,2,sample


In [27]:
schema.Experiment().Sample() & f'username="{username}"' & f'experiment_name="{experiment_name}"'

username user in the lab,experiment_name,sample_name,perfusion_date,expected_handoff_date,clearer netid of person doing the clearing,clearing_protocol,clearing_progress,antibody1,antibody2,imager netid of person doing the imaging,image_resolution,channel488_registration,channel555_registration,channel647_registration,channel790_registration,channel488_injection_detection,channel555_injection_detection,channel647_injection_detection,channel790_injection_detection,channel488_probe_detection,channel555_probe_detection,channel647_probe_detection,channel790_probe_detection,channel488_cell_detection,channel555_cell_detection,channel647_cell_detection,channel790_cell_detection,notes
ahoag,test3,sample-1,None,None,None,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
ahoag,test3,sample-2,None,None,None,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,


In [19]:
schema.Experiment.Sample()

username user in the lab,experiment_name,sample_name,perfusion_date,expected_handoff_date,clearer netid of person doing the clearing,clearing_protocol,clearing_progress,antibody1,antibody2,imager netid of person doing the imaging,image_resolution,channel488_registration,channel555_registration,channel647_registration,channel790_registration,channel488_injection_detection,channel555_injection_detection,channel647_injection_detection,channel790_injection_detection,channel488_probe_detection,channel555_probe_detection,channel647_probe_detection,channel790_probe_detection,channel488_cell_detection,channel555_cell_detection,channel647_cell_detection,channel790_cell_detection,notes
ahoag,test,prefix-1,None,None,ahoag,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,
ahoag,test2,sample-1,None,None,None,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
ahoag,test3,sample-1,None,None,None,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
ahoag,test3,sample-2,None,None,None,iDISCO abbreviated clearing,incomplete,,,None,1.3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,


In [7]:
schema.Experiment()

experiment_id allowed here are sql datatypes.,username user in the lab,labname,correspondence_email,title,description,species,number_of_samples,sample_prefix


In [2]:
# schema = dj.schema('ahoag_lightsheet_demo')
schema = dj.create_virtual_module('ahoag_microscope_demo','ahoag_microscope_demo')

Please enter DataJoint username: ahoag
Please enter DataJoint password: ········
Connecting ahoag@localhost:3306


In [3]:
schema.Microscope().fetch('microscope_name')

array(['light sheet microscope', 'light sheet microscope2', 'test',
       'test2', 'test3', 'test5', 'confocal microscope'], dtype=object)

In [7]:
schema.Channel().fetch("KEY")

[{'microscope_name': 'light sheet microscope', 'channel_name': 'test'},
 {'microscope_name': 'light sheet microscope', 'channel_name': 'test2'}]

In [9]:
schema.Laser().fetch("KEY")[0].keys()

dict_keys(['laser_name'])

In [14]:
schema.Laser()

laser_name,laser_model,laser_serial
Test laser,test model,test serial


In [17]:
insert_dict = {'laser_name':'Test laser','laser_model':'new model','laser_serial':'new serial'}
primary_key_list = schema.Laser().primary_key
primary_key_list

['laser_name']

In [22]:
# primary_key_result_str = ''

primary_key_result_str =','.join(f'{key} = {insert_dict[key]}' for key in primary_key_list)
primary_key_result_str

'laser_name = Test laser'

In [24]:
insert_dict = {'microscope_name':'light sheet microscope','channel_name':'test'}
primary_key_list = schema.Channel().primary_key
primary_key_list

['microscope_name', 'channel_name']

In [26]:
primary_key_result_str =', '.join(f'{key} = {insert_dict[key]}' for key in primary_key_list)
primary_key_result_str

'microscope_name = light sheet microscope, channel_name = test'

In [31]:
schema.Microscope()

microscope_name,center,room_number,optical_bay,loc_on_table,microscope_description
confocal microscope,McDonnell Center,,,,
light sheet microscope,Bezos Center,,,,Lavision Ultramicroscope II
light sheet microscope2,Bezos Center,,,,Future light sheet microscope
test,Bezos Center,asdfadsasdfads,asdfads,asdfadsasdf,asdfadsasdfadsasdfadsasdfadsasdfads
test2,Bezos Center,a,a,a,a
test3,Bezos Center,a,a,a,a
test5,Bezos Center,a,a,a,a


In [289]:
UdiscoClearing().drop()

`ahoag_lightsheet_demo`.`udisco_clearing` (5 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


## 1) Define database tables

In [45]:
@schema
class User(dj.Lookup):
    definition = """
    # Users of the light sheet microscope
    username : varchar(20)      # user in the lab
    ---
    email       : varchar(50)
    """
    
@schema
class Experiment(dj.Manual):
    definition = """ # Experiments performed using the light sheet microscope
    experiment_id           :   smallint auto_increment    # allowed here are sql datatypes.
    ----
    -> User 
    title                   :   varchar(100)
    description             :   varchar(250)
    notes = ""              :   varchar(1000)
    species                 :   varchar(50)
    clearing_protocol       :   enum("iDISCO+_immuno","iDISCO abbreviated clearing","iDISCO abbreviated clearing (rat)","uDISCO","iDISCO_EdU")
    clearing_progress       :   enum("incomplete","complete")
    fluorophores            :   varchar(100)
    antibody1               :   varchar(100)
    antibody2               :   varchar(100)
    image_resolution        :   enum("1.3x","4x")
    cell_detection          :   tinyint
    registration            :   tinyint
    probe_detection         :   tinyint
    injection_detection     :   tinyint
    """  
    
@schema #  
class Microscope(dj.Manual): # dj.Manual is one of the 4 datajoint table types - Manual corresponds to externally inputted data
    definition = """ # Periodic calibration data of the light sheet microscope
    entrynum                :   smallint auto_increment    # allowed here are sql datatypes.
    ----
    -> User              
    date                    :   varchar(10)    
    old_objective           :   varchar(50)
    new_objective           :   varchar(50)
    swapper                 :   varchar(250)
    calibration =           :   varchar(1000) 
    notes =                 :   varchar(1000)
    """

In [46]:
User()

username user in the lab,princeton_email
ahoag,ahoag@princeton.edu


In [47]:
Experiment()

experiment_id allowed here are sql datatypes.,username user in the lab,labname,correspondence_email,title,description,notes,species,clearing_protocol,clearing_progress,fluorophores,antibody1,antibody2,image_resolution,cell_detection,registration,probe_detection,injection_detection
1,ahoag,test,ahoag@princeton.edu,test,test,,mouse,iDISCO abbreviated clearing,in progress,,,,1.3x,0,0,0,0


In [38]:
exp_contents = Experiment() & 'experiment_id=1'

In [40]:
exp_contents.fetch1('cell_detection','registration','probe_detection','injection_detection')

(0, 0, 0, 0)

In [20]:
Microscope()

entrynum allowed here are sql datatypes.,username user in the lab,date,old_objective,new_objective,swapper,calibration,notes


## 2) Download content for the database from existing google sheets and the Sheets API

In [16]:
def retrieve_google_sheet(spreadsheet_id,range_query):
    """
    ---PURPOSE---
    Get data from the range of cells in the google sheet specified.
    ---INPUT---
    spreadsheet_id    The long string in the URL of a google sheet
                      that identifies the sheet
    range_query       The Sheets API range query, e.g. 'Sheet1!A3:B3'
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=range_query).execute()
    values = result.get('values', [])

    return values

## Fill User and Experiment tables

In [17]:
# Grab the form response data so we can fill out User() and Experiment() table
# Set the scope to be read only since we are not adding anything to the google sheet, just reading it
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
# Set the spreadsheet ID (long string in the url, e.g.:
exp_SPREADSHEET_ID = '1iy4H8kHocnu9sS9P-NZA8qcp6_R4gj0ogwksl3I-suU' # The copy I made of the clearing/form responses spreadsheet
exp_range_query='Form Responses!B3:P36'

# https://docs.google.com/spreadsheets/d/1YbP64yIApUaxjpscBXP8r82VmrkrKYakbjRFqF9XHB0/edit#gid=0) 
# As well as the range of cells you want to grab. 
# We are in readonly (see SCOPES) so we cannot destroy it by accident.
secrets_file = 'credentials.json' # Has to be called "credentials.json"  
assert os.path.exists(secrets_file)

In [18]:
def fill_user_exp_tables():
    """ Fills the User() and Experiment() tables 
    using the form response data from the tab in the clearing google spreadsheet
    """

    exp_column_names = ['username','species','clearing_protocol','clearing_progress','title','description','fluorophores',\
                    'antibody1','antibody2','image_resolution',\
                    'cell_detection','registration','probe_detection','injection_detection','notes'] # order doesn't matter since we will be using these in a dictionary
    if len(User.fetch()) != 0 or len(Experiment.fetch()) != 0:
        sys.exit('User and/or Experiment table not empty! Drop them first before refilling')
    form_response_data = retrieve_google_sheet(spreadsheet_id=exp_SPREADSHEET_ID,range_query=exp_range_query)
    # Remove "Number of Samples" column since it is empty and we don't want it
    form_response_data=[x[0:3]+x[4:] for x in form_response_data]
    for form_response in form_response_data:
        # ignore blank lines
        if not form_response:
            continue
        assert len(form_response) == 14
        species = form_response[0].lower()
        clearing_protocol = form_response[1]
        if 'immunostaining' in clearing_protocol:
            clearing_protocol = 'iDISCO+_immuno'
        elif 'abbreviated' in clearing_protocol:
            if species == 'rat':
                clearing_protocol = 'iDISCO abbreviated clearing (rat)'
            else:
                clearing_protocol = 'iDISCO abbreviated clearing'
        elif 'EdU' in clearing_protocol:
            clearing_protocol = 'iDISCO_EdU'
        else: # don't change it
            pass
        clearing_progress = 'complete' # All of the ones in this sheet have already been completed
        title=form_response[2]
        description = form_response[3]
        fluorophores = form_response[4]
        primary_antibody = form_response[5]
        secondary_antibody = form_response[6]
        imaging_str = form_response[7]
        image_resolution = "1.3x" if "1.3x" in imaging_str else "4x"
        processing_str = form_response[8]
        processing_list = [x.lower() for x in processing_str.split(',')]

        cell_detection = 0
        registration = 0
        probe_detection = 0
        injection_detection = 0
        for item in processing_list:
            if 'cell detection' in item:
                cell_detection = 1
            if 'registration' in item:
                registration=1
            if 'probe' in item and 'detection' in item:
                probe_detection = 1
            if 'injection' in item and 'detection' in item:
                injection_detection =1
        notes = form_response[11]
        email = form_response[13]
        username = email.split('@')[0].lower() if 'princeton' in email else 'zmd' # zahra is the only one who used her gmail
        labname = labname_dict[username]
        user_insert_dict = {'username':username,'email':email}
        
        exp_insert_row = [username,species,clearing_protocol,clearing_progress,title,description,fluorophores,\
                      primary_antibody,secondary_antibody,image_resolution,cell_detection,registration,\
                      probe_detection,injection_detection,notes]
        exp_insert_dict = {exp_column_names[ii]:exp_insert_row[ii] for ii in range(len(exp_column_names))}
        
        User().insert1(user_insert_dict,skip_duplicates=True)
        Experiment().insert1(exp_insert_dict)

In [19]:
exp_response_data = retrieve_google_sheet(spreadsheet_id=exp_SPREADSHEET_ID,range_query=exp_range_query)

In [20]:
fill_user_exp_tables()

NameError: name 'User' is not defined

In [257]:
def pickle_exp_data():
    today = datetime.now().strftime('%Y-%m-%d')
    exp_response_data = retrieve_google_sheet(spreadsheet_id=exp_SPREADSHEET_ID,range_query=exp_range_query)
    experiment_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/experiment_data_{today}.pkl'
    with open(experiment_data_file, 'wb') as f:
        pickle.dump(exp_response_data,f)
        print(f"Wrote {experiment_data_file}")

In [258]:
def pickle_exp_test_data():
    exp_response_data = retrieve_google_sheet(spreadsheet_id=exp_SPREADSHEET_ID,range_query=exp_range_query)
    experiment_test_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/experiment_test_data.pkl'
    with open(experiment_test_data_file, 'wb') as f:
        pickle.dump(exp_response_data,f)
        print(f"Wrote {experiment_test_data_file}")

In [14]:
experiment_test_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/experiment_test_data.pkl'
with open(experiment_test_data_file,'rb') as f:
    data = pickle.load(f)

In [21]:
labname_dict = {'tomohito':'Falkner','jverpeut':'Wang','soline':'Falkner','marlies.oostland':'Wang','zmd':'Wang'}

In [23]:
len(labname_dict)

5

In [15]:
data

[['Mouse',
  'uDISCO',
  'tomo_vg5_uDISCO_iDISCO_test_20190602',
  '',
  'visualize GCaMP mCherry and transgenic GFP expression without immunostaining',
  'mCherry (endogenous), GFP (endogenous)',
  '',
  '',
  '1.3x (low-resolution, good for site detection, whole brain c-fos quantification, or registration)',
  '',
  '',
  '',
  '',
  '',
  'tomohito@princeton.edu'],
 ['Mouse',
  'iDISCO+ (immunostaining)',
  'DREADDymaze',
  '',
  'Image c-fos in whole brains at 1.3x.',
  'None',
  'Anti cfos 1:1000',
  'Donkey antirabbit AF647 1:200',
  '1.3x (low-resolution, good for site detection, whole brain c-fos quantification, or registration)',
  'Registration, Cell detection',
  '',
  '',
  '',
  'Jess (Wang)',
  'jverpeut@princeton.edu'],
 [],
 ['Mouse',
  'iDISCO for non-oxidizable fluorophores (abbreviated clearing)',
  '20190304_LW_m340',
  '',
  'Detect placement of optical fiber tip',
  'GCampxDat-Cre Double Transgentic Mouse',
  'None',
  'None',
  '1.3x (low-resolution, good for sit

In [282]:
udisco_test_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/udisco_test_data.pkl'
with open(udisco_test_data_file,'rb') as f:
    data = pickle.load(f)

In [283]:
df = pd.DataFrame(data)
# df = df.set_index('experiment_id')
df.to_dict('r')

[{'experiment_id': 1,
  'username': 'tomohito',
  'clearer': 'ahoag',
  'exp_notes': 'Half brain',
  'perfusion_date': None,
  'time_dehydr_pbs_wash1': Timestamp('2019-06-02 16:35:00'),
  'dehydr_pbs_wash1_notes': '',
  'time_dehydr_butanol_30percent': Timestamp('2019-06-03 14:15:00'),
  'dehydr_butanol_30percent_notes': '',
  'time_dehydr_butanol_50percent': Timestamp('2019-06-04 11:06:00'),
  'dehydr_butanol_50percent_notes': '',
  'time_dehydr_butanol_70percent': Timestamp('2019-06-04 18:15:00'),
  'dehydr_butanol_70percent_notes': '',
  'time_dehydr_butanol_80percent': Timestamp('2019-06-05 10:14:00'),
  'dehydr_butanol_80percent_notes': '',
  'time_dehydr_butanol_90percent': Timestamp('2019-06-05 18:02:00'),
  'dehydr_butanol_90percent_notes': '',
  'time_dehydr_butanol_96percent': Timestamp('2019-06-06 10:10:00'),
  'dehydr_butanol_96percent_notes': '',
  'time_dehydr_butanol_100percent': Timestamp('2019-06-06 18:52:00'),
  'dehydr_butanol_100percent_notes': '',
  'time_clearing_

In [230]:
fill_user_exp_tables()

In [232]:
User()

username user in the lab,email
apv2,apv2@princeton.edu
ejdennis,Ejdennis@princeton.edu
jverpeut,jverpeut@princeton.edu
marlies.oostland,marlies.oostland@princeton.edu
mioffe,mioffe@princeton.edu
pbibawi,pbibawi@princeton.edu
rmallarino,rmallarino@princeton.edu
soline,soline@princeton.edu
tomohito,tomohito@princeton.edu
willmore,Willmore@princeton.edu


In [233]:
Experiment()

experiment_id allowed here are sql datatypes.,username user in the lab,title,description,notes,species,clearing_protocol,clearing_progress,fluorophores,antibody1,antibody2,image_resolution,cell_detection,registration,probe_detection,injection_detection
1,tomohito,tomo_vg5_uDISCO_iDISCO_test_20190602,visualize GCaMP mCherry and transgenic GFP expression without immunostaining,,mouse,uDISCO,complete,"mCherry (endogenous), GFP (endogenous)",,,1.3x,0,0,0,0
2,jverpeut,DREADDymaze,Image c-fos in whole brains at 1.3x.,,mouse,iDISCO+_immuno,complete,None,Anti cfos 1:1000,Donkey antirabbit AF647 1:200,1.3x,1,1,0,0
3,willmore,20190304_LW_m340,Detect placement of optical fiber tip,,mouse,iDISCO abbreviated clearing,complete,GCampxDat-Cre Double Transgentic Mouse,None,None,1.3x,0,1,1,0
4,jverpeut,an1-31,Recover mcherry expression in cerebellum. Morphology analysis.,,mouse,iDISCO+_immuno,complete,mcherry,rabbit anti-RFP 1:1000,Donkey antiRabbit 647 1:450,1.3x,0,1,0,1
5,apv2,20190313_IBL_DiI_1,Detection of CM DiI,,mouse,iDISCO abbreviated clearing,complete,CM DiI,NA,NA,4x,0,0,1,0
6,zmd,"20181217_marlies, 20181031_marlies_M12",recovery of electrode tract,Pilot,mouse,iDISCO abbreviated clearing,complete,N/A,N/A,N/A,1.3x,0,1,1,0
7,jverpeut,EAAT4- 14 samples,Look for cerebellar banding. GFP,,mouse,iDISCO+_immuno,complete,GFP,Chicken antiGFP Aves 1:1000,Donkey anti chicken 647 1:750,1.3x,1,1,0,0
8,mioffe,a1_20190327 a2_20190327 a3_20190327,"To check expression of gcamp in two animals (a1 and a2), and one opsin tagged with gfp (a3)",,mouse,iDISCO+_immuno,complete,"gcamp in a1 and a2, gfp in a3, which was injected with soCoChR-GFP",I don't know..,I don't know..,1.3x,0,1,0,1
9,jverpeut,LindsayCrusI_ymaze_cfos (25 samples),Detect c-fos.,,mouse,iDISCO+_immuno,complete,AlexaFluor 647,SySy Rb antiFos 1:1000,Donkey antiRabbit 647 1:200,1.3x,1,1,0,0
10,jverpeut,AdultChronicD_MLI_Lawrence (1-12 each batch),detect c-Fos,12 LobVI and 12 crusI,mouse,iDISCO+_immuno,complete,AlexaFluor 647,SySy Rb antiFos 1:1000,Donkey antiRabbit 647 1:200,1.3x,1,1,0,0


## Fill Microscope Table

In [17]:
# Only does this if the table does not already exist.
def fill_microscope_table():
    microscope_spreadsheet_id = '1Velwahzvy0hxHDlcjlQPbLjShCQMmCb2DWuz-O_65PI' # The live spreadsheet
    microscope_range_query = 'Sheet1' # Syntax is Sheet_name!cell_start:cell_end, e.g. 
    if len(Microscope.fetch()) != 0:
        sys.exit('Microscope table not empty. Drop it before attempting to refill')
    microscope_data = retrieve_google_sheet(spreadsheet_id=microscope_spreadsheet_id,range_query=microscope_range_query)
    microscope_column_names = ['username','date','old_objective','new_objective','swapper','calibration','notes'] # in the database
    for row in microscope_data[1:]: # skip the first row because it is the column names
        if len(row) < 2:
            continue
        try:
            date_str = datetime.strptime(row[0], "%B %d, %Y").strftime("%Y-%m-%d")
        except:
            date_str = 'unknown'
        for ii in range(len(microscope_column_names)-len(row)):
            row.append('')
        microscope_insert_row = ['zmd',date_str] + row[1:]    
        microscope_insert_dict = {microscope_column_names[ii]:microscope_insert_row[ii]\
                                  for ii in range(len(microscope_column_names))}
        Microscope().insert1(microscope_insert_dict,skip_duplicates = True)

In [23]:
fill_microscope_table()

In [35]:
today = datetime.now().strftime('%Y-%m-%d')
microscope_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/microscope_data_{today}.pkl'
with open(microscope_data_file, 'wb') as f:
    pickle.dump(microscope_data_file,f)
    print(f"wrote {microscope_data_file}")

wrote /home/ahoag/Git/lightserv_dj_demo/lightserv/data/microscope_data_2019-10-01.pkl


In [24]:
Microscope()

entrynum allowed here are sql datatypes.,username user in the lab,date,old_objective,new_objective,swapper,calibration,notes
1,zmd,2018-02-20,1.3x,4x,"Tom Pisano, Ben Deverett, Stephan Thiberge",,
2,zmd,2018-03-09,4x,1.3x,"Tom Pisano, Kelly Seagraves",,
3,zmd,2018-03-13,1.3x,4x,"Tom Pisano, Kelly Seagraves",,
4,zmd,2018-03-20,4x,4x,"Tom Pisano, Kelly Seagraves",Calibration: Bilateral individual sheets relative to each other. Chromatic correction.,
5,zmd,2018-03-29,4x,4x,"Tom Pisano, Stephan",Calibration: Bilateral individual sheets relative to each other. Chromatic correction.,
6,zmd,2018-04-11,4x,4x,Tom Pisano,Calibration: Bilateral individual sheets relative to each other.,Within sheets OK. Left drifted relative to R
7,zmd,2018-04-23,4x,4x,Tom Pisano,Calibration: Bilateral individual sheets relative to each other.,Left sheet downward beam seems slightly off. Pretty good intersheets
8,zmd,2018-04-30,4x,1.3x,Tom Pisano,Calibration: Bilateral individual sheets relative to each other.,Left sheet downward beam seems slightly off. Pretty good intersheets (except for R wall most beam)
9,zmd,2018-05-07,1.3x,4x,Tom Pisano,Calibration: Bilateral individual sheets relative to each other.,Fairly OK except for R upward pointing sheet. Seems really off
10,zmd,2018-07-10,1.3x,4x,"Tom, Zahra","Calibration, Left relative to right. Seems that the right upward pointing beam is drifting as well as gross L relative to R",


## Add clearing tables

### iDISCO+ clearing table

In [11]:
@schema
class IdiscoPlusClearing(dj.Manual): # dj.Manual is one of the 4 datajoint table types - Manual corresponds to externally inputted data
        definition = """ # Periodic calibration data of the light sheet microscope
        -> Experiment              # experiment_id, the primary key from the Experiment() table
        ----
        -> User                    # username, the researcher's netid from the User() table
        clearer                                                  :   varchar(20)   # the netid of the person who did the clearing
        exp_notes = ""                                           :   varchar(500)  # Note anything unusual that happened during experiment that could affect clearing
        perfusion_date = NULL                                    :   date 
        time_dehydr_pbs_wash1 = NULL                             :   datetime
        dehydr_pbs_wash1_notes = ""                              :   varchar(250)
        time_dehydr_pbs_wash2 = NULL                             :   datetime
        dehydr_pbs_wash2_notes = ""                              :   varchar(250)
        time_dehydr_pbs_wash3 = NULL                             :   datetime
        dehydr_pbs_wash3_notes = ""                              :   varchar(250)
        time_dehydr_ch3oh_20percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_20percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_40percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_40percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_60percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_60percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_80percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_80percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_100percent_wash1 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash1_notes = ""                 :   varchar(250)
        time_dehydr_ch3oh_100percent_wash2 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash2_notes = ""                 :   varchar(250)
        time_dehydr_h202_wash1 = NULL                            :   datetime
        dehydr_h202_wash1_notes = ""                             :   varchar(250)
        time_rehydr_ch3oh_100percent_wash1 = NULL                :   datetime
        rehydr_ch3oh_100percent_wash1_notes = ""                 :   varchar(250)
        time_rehydr_ch3oh_80percent_wash1 = NULL                 :   datetime
        rehydr_ch3oh_80percent_wash1_notes = ""                  :   varchar(250)
        time_rehydr_ch3oh_60percent_wash1 = NULL                 :   datetime
        rehydr_ch3oh_60percent_wash1_notes = ""                  :   varchar(250)
        time_rehydr_ch3oh_40percent_wash1 = NULL                 :   datetime
        rehydr_ch3oh_40percent_wash1_notes = ""                  :   varchar(250)
        time_rehydr_ch3oh_20percent_wash1 = NULL                 :   datetime
        rehydr_ch3oh_20percent_wash1_notes = ""                  :   varchar(250)
        time_rehydr_pbs_wash1 = NULL                             :   datetime
        rehydr_pbs_wash1_notes = ""                              :   varchar(250)
        time_rehydr_sodium_azide_wash1 = NULL                    :   datetime
        rehydr_sodium_azide_wash1_notes = ""                     :   varchar(250)
        time_rehydr_sodium_azide_wash2 = NULL                    :   datetime
        rehydr_sodium_azide_wash2_notes = ""                     :   varchar(250)
        time_rehydr_glycine_wash1 = NULL                         :   datetime
        rehydr_glycine_wash1_notes = ""                          :   varchar(250)
        time_blocking_start_roomtemp = NULL                      :   datetime
        blocking_start_roomtemp_notes = ""                       :   varchar(250)
        time_blocking_donkey_serum = NULL                        :   datetime
        blocking_donkey_serum_notes = ""                         :   varchar(250)
        time_antibody1_start_roomtemp = NULL                     :   datetime
        antibody1_start_roomtemp_notes = ""                      :   varchar(250)
        time_antibody1_ptwh_wash1 = NULL                         :   datetime
        antibody1_ptwh_wash1_notes = ""                          :   varchar(250)
        time_antibody1_ptwh_wash2 = NULL                         :   datetime
        antibody1_ptwh_wash2_notes = ""                          :   varchar(250)
        time_antibody1_added = NULL                              :   datetime
        antibody1_added_notes = ""                               :   varchar(250)
        time_wash1_start_roomtemp = NULL                         :   datetime
        wash1_start_roomtemp_notes = ""                          :   varchar(250)
        time_wash1_ptwh_wash1 = NULL                             :   datetime
        wash1_ptwh_wash1_notes = ""                              :   varchar(250)
        time_wash1_ptwh_wash2 = NULL                             :   datetime
        wash1_ptwh_wash2_notes = ""                              :   varchar(250)
        time_wash1_ptwh_wash3 = NULL                             :   datetime
        wash1_ptwh_wash3_notes = ""                              :   varchar(250)
        time_wash1_ptwh_wash4 = NULL                             :   datetime
        wash1_ptwh_wash4_notes = ""                              :   varchar(250)
        time_wash1_ptwh_wash5 = NULL                             :   datetime
        wash1_ptwh_wash5_notes = ""                              :   varchar(250)
        time_antibody2_added = NULL                              :   datetime
        antibody2_added_notes = ""                               :   varchar(250)
        time_wash2_start_roomtemp = NULL                         :   datetime
        wash2_start_roomtemp_notes = ""                          :   varchar(250)
        time_wash2_ptwh_wash1 = NULL                             :   datetime
        wash2_ptwh_wash1_notes = ""                              :   varchar(250)
        time_wash2_ptwh_wash2 = NULL                             :   datetime
        wash2_ptwh_wash2_notes = ""                              :   varchar(250)
        time_wash2_ptwh_wash3 = NULL                             :   datetime
        wash2_ptwh_wash3_notes = ""                              :   varchar(250)
        time_wash2_ptwh_wash4 = NULL                             :   datetime
        wash2_ptwh_wash3_notes = ""                              :   varchar(250)
        time_wash2_ptwh_wash5 = NULL                             :   datetime
        wash2_ptwh_wash5_notes = ""                              :   varchar(250)
        time_clearing_ch3oh_20percent_wash1 = NULL               :   datetime
        clearing_ch3oh_20percent_wash1_notes = ""                :   varchar(250)
        time_clearing_ch3oh_40percent_wash1 = NULL               :   datetime
        clearing_ch3oh_40percent_wash1_notes = ""                :   varchar(250)
        time_clearing_ch3oh_60percent_wash1 = NULL               :   datetime
        clearing_ch3oh_60percent_wash1_notes = ""                :   varchar(250)
        time_clearing_ch3oh_80percent_wash1 = NULL               :   datetime
        clearing_ch3oh_80percent_wash1_notes = ""                :   varchar(250)
        time_clearing_ch3oh_100percent_wash1 = NULL              :   datetime
        clearing_ch3oh_100percent_wash1_notes = ""               :   varchar(250)
        time_clearing_ch3oh_100percent_wash2 = NULL              :   datetime
        clearing_ch3oh_100percent_wash2_notes = ""               :   varchar(250)
        time_clearing_dcm_66percent_ch3oh_33percent = NULL       :   datetime
        clearing_dcm_66percent_ch3oh_33percent_notes = ""        :   varchar(250)
        time_clearing_dcm_wash1 = NULL                           :   datetime
        clearing_dcm_wash1_notes = ""                            :   varchar(250)
        time_clearing_dcm_wash2 = NULL                           :   datetime
        clearing_dcm_wash2_notes = ""                            :   varchar(250)
        time_clearing_dbe = NULL                                 :   datetime
        clearing_dbe_notes = ""                                  :   varchar(250)
        time_clearing_new_tubes = NULL                           :   datetime
        clearing_new_tubes_notes = ""                            :   varchar(250)
        clearing_notes = ""                                      :   varchar(500)
        """

In [12]:
@schema
class IdiscoAbbreviatedClearing(dj.Manual): # dj.Manual is one of the 4 datajoint table types - Manual corresponds to externally inputted data
        definition = """ # Periodic calibration data of the light sheet microscope
        -> Experiment              # experiment_id, the primary key from the Experiment() table
        ----
        -> User                    # username, the researcher's netid from the User() table
        clearer                                                  :   varchar(20)   # the netid of the person who did the clearing
        exp_notes = ""                                           :   varchar(500)  # Note anything unusual that happened during experiment that could affect clearing
        perfusion_date = NULL                                    :   date 
        time_pbs_wash1 = NULL                                    :   datetime
        pbs_wash1_notes = ""                                     :   varchar(250)
        time_pbs_wash2 = NULL                                    :   datetime
        pbs_wash2_notes = ""                                     :   varchar(250)
        time_pbs_wash3 = NULL                                    :   datetime
        pbs_wash3_notes = ""                                     :   varchar(250)
        time_dehydr_ch3oh_20percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_20percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_40percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_40percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_60percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_60percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_80percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_80percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_100percent_wash1 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash1_notes = ""                 :   varchar(250)
        time_dehydr_ch3oh_100percent_wash2 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash2_notes = ""                 :   varchar(250)
        time_dehydr_dcm_66percent_ch3oh_33percent = NULL         :   datetime
        dehydr_dcm_66percent_ch3oh_33percent_notes = ""          :   varchar(250)
        time_dehydr_dcm_wash1 = NULL                             :   datetime
        dehydr_dcm_wash1_notes = ""                              :   varchar(250)
        time_dehydr_dcm_wash2 = NULL                             :   datetime
        dehydr_dcm_wash2_notes = ""                              :   varchar(250)
        time_dehydr_dbe_wash1 = NULL                             :   datetime
        dehydr_dbe_wash1_notes = ""                              :   varchar(250)
        time_dehydr_dbe_wash2 = NULL                             :   datetime
        dehydr_dbe_wash2_notes = ""                              :   varchar(250)
        clearing_notes = ""                                      :   varchar(500)
        """

In [13]:
@schema
class IdiscoAbbreviatedRatClearing(dj.Manual): # dj.Manual is one of the 4 datajoint table types - Manual corresponds to externally inputted data
        definition = """ # Abbreviated Rat clearing table
        -> Experiment              # experiment_id, the primary key from the Experiment() table
        ----
        -> User                    # username, the researcher's netid from the User() table
        clearer                                                  :   varchar(20)   # the netid of the person who did the clearing
        exp_notes = ""                                           :   varchar(500)  # Note anything unusual that happened during experiment that could affect clearing
        perfusion_date = NULL                                    :   date 
        time_pbs_wash1 = NULL                                    :   datetime
        pbs_wash1_notes = ""                                     :   varchar(250)
        time_pbs_wash2 = NULL                                    :   datetime
        pbs_wash2_notes = ""                                     :   varchar(250)
        time_pbs_wash3 = NULL                                    :   datetime
        pbs_wash3_notes = ""                                     :   varchar(250)
        time_dehydr_ch3oh_20percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_20percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_40percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_40percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_60percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_60percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_80percent_wash1 = NULL                 :   datetime
        dehydr_ch3oh_80percent_wash1_notes = ""                  :   varchar(250)
        time_dehydr_ch3oh_100percent_wash1 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash1_notes = ""                 :   varchar(250)
        time_dehydr_ch3oh_100percent_wash2 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash2_notes = ""                 :   varchar(250)
        time_dehydr_h202_wash1 = NULL                            :   datetime
        dehydr_h202_wash1_notes = ""                             :   varchar(250)
        time_dehydr_ch3oh_100percent_wash3 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash3_notes = ""                 :   varchar(250)
        time_dehydr_ch3oh_100percent_wash4 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash4_notes = ""                 :   varchar(250)
        time_dehydr_ch3oh_100percent_wash5 = NULL                :   datetime
        dehydr_ch3oh_100percent_wash5_notes = ""                 :   varchar(250)
        time_dehydr_dcm_66percent_ch3oh_33percent = NULL         :   datetime
        dehydr_dcm_66percent_ch3oh_33percent_notes = ""          :   varchar(250)
        time_dehydr_dcm_wash1 = NULL                             :   datetime
        dehydr_dcm_wash1_notes = ""                              :   varchar(250)
        time_dehydr_dcm_wash2 = NULL                             :   datetime
        dehydr_dcm_wash2_notes = ""                              :   varchar(250)
        time_dehydr_dbe_wash1 = NULL                             :   datetime
        dehydr_dbe_wash1_notes = ""                              :   varchar(250)
        time_dehydr_dbe_wash2 = NULL                             :   datetime
        dehydr_dbe_wash2_notes = ""                              :   varchar(250)
        clearing_notes = ""                                      : varchar(500)
        """

In [48]:
@schema
class UdiscoClearing(dj.Manual): # dj.Manual is one of the 4 datajoint table types - Manual corresponds to externally inputted data
    definition = """ # uDISCO clearing table
    -> Experiment              # experiment_id, the primary key from the Experiment() table
    ----
    -> User                    # username, the researcher's netid from the User() table
    clearer                                                  :   varchar(20)   # the netid of the person who did the clearing
    exp_notes = ""                                           :   varchar(500)  # Note anything unusual that happened during experiment that could affect clearing
    perfusion_date = NULL                                    :   date 
    time_dehydr_pbs_wash1 = NULL                             :   datetime
    dehydr_pbs_wash1_notes = ""                              :   varchar(250)
    time_dehydr_butanol_30percent = NULL                     :   datetime
    dehydr_butanol_30percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_50percent = NULL                     :   datetime
    dehydr_butanol_50percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_70percent = NULL                     :   datetime
    dehydr_butanol_70percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_80percent = NULL                     :   datetime
    dehydr_butanol_80percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_90percent = NULL                     :   datetime
    dehydr_butanol_90percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_96percent = NULL                     :   datetime
    dehydr_butanol_96percent_notes = ""                      :   varchar(250)
    time_dehydr_butanol_100percent = NULL                    :   datetime
    dehydr_butanol_100percent_notes = ""                     :   varchar(250)
    time_clearing_dcm_wash1 = NULL                           :   datetime
    clearing_dcm_wash1_notes = ""                            :   varchar(250)
    time_clearing_babb_wash1 = NULL                          :   datetime
    clearing_babb_wash1_notes = ""                           :   varchar(250)
    clearing_notes = ""                                      : varchar(500)
    """

## Pickle data currently in the database

In [244]:
def pickle_udisco_data():
    udisco_data = UdiscoClearing().fetch()
    # pickle current udisco data
    today = datetime.now().strftime('%Y-%m-%d')
    udisco_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/udisco_data_{today}.pkl'
    udisco_data_file_latest = home_dir + '/Git/lightserv_dj_demo/lightserv/data/udisco_data_latest.pkl'
    with open(udisco_data_file, 'wb') as f:
        pickle.dump(udisco_data,f)
        print(f"wrote {udisco_data_file}")
    copyfile(udisco_data_file,udisco_data_file_latest)

In [33]:
def pickle_idiscoplus_data():
    idiscoplus_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/idiscoplus_data_latest.pkl'
    with open(idiscoplus_data_file, 'wb') as f:
        pickle.dump(idiscoplus_data,f)
        print(f"wrote {idiscoplus_data_file}")

In [34]:
pickle_idiscoplus_data()

wrote /home/ahoag/Git/lightserv_dj_demo/lightserv/data/idiscoplus_data_latest.pkl


## Fill clearing tables 

In [296]:
fill_uDISCO_table()

In [49]:
UdiscoClearing()

experiment_id allowed here are sql datatypes.,username user in the lab,clearer the netid of the person who did the clearing,exp_notes Note anything unusual that happened during experiment that could affect clearing,perfusion_date,time_dehydr_pbs_wash1,dehydr_pbs_wash1_notes,time_dehydr_butanol_30percent,dehydr_butanol_30percent_notes,time_dehydr_butanol_50percent,dehydr_butanol_50percent_notes,time_dehydr_butanol_70percent,dehydr_butanol_70percent_notes,time_dehydr_butanol_80percent,dehydr_butanol_80percent_notes,time_dehydr_butanol_90percent,dehydr_butanol_90percent_notes,time_dehydr_butanol_96percent,dehydr_butanol_96percent_notes,time_dehydr_butanol_100percent,dehydr_butanol_100percent_notes,time_clearing_dcm_wash1,clearing_dcm_wash1_notes,time_clearing_babb_wash1,clearing_babb_wash1_notes,clearing_notes


In [19]:
IdiscoPlusClearing()

experiment_id allowed here are sql datatypes.,username user in the lab,clearer the netid of the person who did the clearing,exp_notes Note anything unusual that happened during experiment that could affect clearing,perfusion_date,time_dehydr_pbs_wash1,dehydr_pbs_wash1_notes,time_dehydr_pbs_wash2,dehydr_pbs_wash2_notes,time_dehydr_pbs_wash3,dehydr_pbs_wash3_notes,time_dehydr_ch3oh_20percent_wash1,dehydr_ch3oh_20percent_wash1_notes,time_dehydr_ch3oh_40percent_wash1,dehydr_ch3oh_40percent_wash1_notes,time_dehydr_ch3oh_60percent_wash1,dehydr_ch3oh_60percent_wash1_notes,time_dehydr_ch3oh_80percent_wash1,dehydr_ch3oh_80percent_wash1_notes,time_dehydr_ch3oh_100percent_wash1,dehydr_ch3oh_100percent_wash1_notes,time_dehydr_ch3oh_100percent_wash2,dehydr_ch3oh_100percent_wash2_notes,time_dehydr_h202_wash1,dehydr_h202_wash1_notes,time_rehydr_ch3oh_100percent_wash1,rehydr_ch3oh_100percent_wash1_notes,time_rehydr_ch3oh_80percent_wash1,rehydr_ch3oh_80percent_wash1_notes,time_rehydr_ch3oh_60percent_wash1,rehydr_ch3oh_60percent_wash1_notes,time_rehydr_ch3oh_40percent_wash1,rehydr_ch3oh_40percent_wash1_notes,time_rehydr_ch3oh_20percent_wash1,rehydr_ch3oh_20percent_wash1_notes,time_rehydr_pbs_wash1,rehydr_pbs_wash1_notes,time_rehydr_sodium_azide_wash1,rehydr_sodium_azide_wash1_notes,time_rehydr_sodium_azide_wash2,rehydr_sodium_azide_wash2_notes,time_rehydr_glycine_wash1,rehydr_glycine_wash1_notes,time_blocking_start_roomtemp,blocking_start_roomtemp_notes,time_blocking_donkey_serum,blocking_donkey_serum_notes,time_antibody1_start_roomtemp,antibody1_start_roomtemp_notes,time_antibody1_ptwh_wash1,antibody1_ptwh_wash1_notes,time_antibody1_ptwh_wash2,antibody1_ptwh_wash2_notes,time_antibody1_added,antibody1_added_notes,time_wash1_start_roomtemp,wash1_start_roomtemp_notes,time_wash1_ptwh_wash1,wash1_ptwh_wash1_notes,time_wash1_ptwh_wash2,wash1_ptwh_wash2_notes,time_wash1_ptwh_wash3,wash1_ptwh_wash3_notes,time_wash1_ptwh_wash4,wash1_ptwh_wash4_notes,time_wash1_ptwh_wash5,wash1_ptwh_wash5_notes,time_antibody2_added,antibody2_added_notes,time_wash2_start_roomtemp,wash2_start_roomtemp_notes,time_wash2_ptwh_wash1,wash2_ptwh_wash1_notes,time_wash2_ptwh_wash2,wash2_ptwh_wash2_notes,time_wash2_ptwh_wash3,wash2_ptwh_wash3_notes,time_wash2_ptwh_wash4,time_wash2_ptwh_wash5,wash2_ptwh_wash5_notes,time_clearing_ch3oh_20percent_wash1,clearing_ch3oh_20percent_wash1_notes,time_clearing_ch3oh_40percent_wash1,clearing_ch3oh_40percent_wash1_notes,time_clearing_ch3oh_60percent_wash1,clearing_ch3oh_60percent_wash1_notes,time_clearing_ch3oh_80percent_wash1,clearing_ch3oh_80percent_wash1_notes,time_clearing_ch3oh_100percent_wash1,clearing_ch3oh_100percent_wash1_notes,time_clearing_ch3oh_100percent_wash2,clearing_ch3oh_100percent_wash2_notes,time_clearing_dcm_66percent_ch3oh_33percent,clearing_dcm_66percent_ch3oh_33percent_notes,time_clearing_dcm_wash1,clearing_dcm_wash1_notes,time_clearing_dcm_wash2,clearing_dcm_wash2_notes,time_clearing_dbe,clearing_dbe_notes,time_clearing_new_tubes,clearing_new_tubes_notes,clearing_notes


In [35]:
idiscoplus_data_file = home_dir + '/Git/lightserv_dj_demo/lightserv/data/idiscoplus_data_latest.pkl'
with open(idiscoplus_data_file,'rb') as f:
    idiscoplus_data = pickle.load(f)

In [46]:
udisco_data_file = home_dir + '/Git/lightserv_dj_demo/lightserv/data/udisco_data_latest.pkl'
with open(udisco_data_file,'rb') as f:
    udisco_data = pickle.load(f)
udisco_data

array([( 1, 'tomohito', 'ahoag', 'Half brain', None, datetime.datetime(2019, 6, 2, 16, 35), '', datetime.datetime(2019, 6, 3, 14, 15), '', datetime.datetime(2019, 6, 4, 11, 6), '', datetime.datetime(2019, 6, 4, 18, 15), '', datetime.datetime(2019, 6, 5, 10, 14), '', datetime.datetime(2019, 6, 5, 18, 2), '', datetime.datetime(2019, 6, 6, 10, 10), '', datetime.datetime(2019, 6, 6, 18, 52), '', datetime.datetime(2019, 6, 7, 9, 27), '', datetime.datetime(2019, 6, 7, 10, 29), '', ''),
       (14, 'mioffe', 'ahoag', '', None, None, '', datetime.datetime(2019, 6, 12, 12, 28), '', datetime.datetime(2019, 6, 13, 10, 43), '', datetime.datetime(2019, 6, 13, 17, 25), '', datetime.datetime(2019, 6, 14, 9, 39), '', datetime.datetime(2019, 6, 17, 9, 52), '', datetime.datetime(2019, 6, 17, 18, 6), '', datetime.datetime(2019, 6, 18, 9, 55), '', datetime.datetime(2019, 6, 19, 9, 22), 'RT 9:22 AM; DCM 12:15 PM', datetime.datetime(2019, 6, 19, 13, 44), '', '')],
      dtype=[('experiment_id', '<i8'), ('us

In [49]:
udisco_data[1]

(15, 'mioffe', 'ahoag', '', None, None, '', datetime.datetime(2019, 6, 12, 12, 28), '', datetime.datetime(2019, 6, 13, 10, 43), '', datetime.datetime(2019, 6, 13, 17, 25), '', datetime.datetime(2019, 6, 14, 9, 39), '', datetime.datetime(2019, 6, 17, 9, 52), '', datetime.datetime(2019, 6, 17, 18, 6), '', datetime.datetime(2019, 6, 18, 9, 55), '', datetime.datetime(2019, 6, 19, 9, 22), 'RT 9:22 AM; DCM 12:15 PM', datetime.datetime(2019, 6, 19, 13, 44), '', '')

In [60]:
# Only does this if the table does not already exist.
def fill_uDISCO_table():
    today = datetime.now().strftime('%Y-%m-%d')
    udisco_data_file_latest = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/udisco_data_latest.pkl'
    with open(udisco_data_file_latest,'rb') as f:
        udisco_data = pickle.load(f)
    for row in udisco_data:
        UdiscoClearing().insert1(row)

In [58]:
UdiscoClearing().delete_quick()

In [61]:
fill_uDISCO_table()

In [41]:
UdiscoClearing()

NameError: name 'UdiscoClearing' is not defined

In [63]:
udisco_data

array([( 1, 'tomohito', 'ahoag', 'Half brain', None, datetime.datetime(2019, 6, 2, 16, 35), '', datetime.datetime(2019, 6, 3, 14, 15), '', datetime.datetime(2019, 6, 4, 11, 6), '', datetime.datetime(2019, 6, 4, 18, 15), '', datetime.datetime(2019, 6, 5, 10, 14), '', datetime.datetime(2019, 6, 5, 18, 2), '', datetime.datetime(2019, 6, 6, 10, 10), '', datetime.datetime(2019, 6, 6, 18, 52), '', datetime.datetime(2019, 6, 7, 9, 27), '', datetime.datetime(2019, 6, 7, 10, 29), '', ''),
       (15, 'mioffe', 'ahoag', '', None, None, '', datetime.datetime(2019, 6, 12, 12, 28), '', datetime.datetime(2019, 6, 13, 10, 43), '', datetime.datetime(2019, 6, 13, 17, 25), '', datetime.datetime(2019, 6, 14, 9, 39), '', datetime.datetime(2019, 6, 17, 9, 52), '', datetime.datetime(2019, 6, 17, 18, 6), '', datetime.datetime(2019, 6, 18, 9, 55), '', datetime.datetime(2019, 6, 19, 9, 22), 'RT 9:22 AM; DCM 12:15 PM', datetime.datetime(2019, 6, 19, 13, 44), '', '')],
      dtype=[('experiment_id', '<i8'), ('us

In [64]:
today = datetime.now().strftime('%Y-%m-%d')
udisco_test_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/udisco_test_data.pkl'
udisco_data_file = home_dir + f'/Git/lightserv_dj_demo/lightserv/data/udisco_data_{today}.pkl'
udisco_data_file_latest = home_dir + '/Git/lightserv_dj_demo/lightserv/data/udisco_data_latest.pkl'
with open(udisco_test_file, 'wb') as f:
    pickle.dump(udisco_data,f)
    print(f"wrote {udisco_test_file}")


wrote /home/ahoag/Git/lightserv_dj_demo/lightserv/data/udisco_test_data.pkl


In [65]:
IdiscoAbbreviatedClearing()

experiment_id allowed here are sql datatypes.,username user in the lab,clearer the netid of the person who did the clearing,exp_notes Note anything unusual that happened during experiment that could affect clearing,perfusion_date,time_pbs_wash1,pbs_wash1_notes,time_pbs_wash2,pbs_wash2_notes,time_pbs_wash3,pbs_wash3_notes,time_dehydr_ch3oh_20percent_wash1,dehydr_ch3oh_20percent_wash1_notes,time_dehydr_ch3oh_40percent_wash1,dehydr_ch3oh_40percent_wash1_notes,time_dehydr_ch3oh_60percent_wash1,dehydr_ch3oh_60percent_wash1_notes,time_dehydr_ch3oh_80percent_wash1,dehydr_ch3oh_80percent_wash1_notes,time_dehydr_ch3oh_100percent_wash1,dehydr_ch3oh_100percent_wash1_notes,time_dehydr_ch3oh_100percent_wash2,dehydr_ch3oh_100percent_wash2_notes,time_dehydr_dcm_66percent_ch3oh_33percent,dehydr_dcm_66percent_ch3oh_33percent_notes,time_dehydr_dcm_wash1,dehydr_dcm_wash1_notes,time_dehydr_dcm_wash2,dehydr_dcm_wash2_notes,time_dehydr_dbe_wash1,dehydr_dbe_wash1_notes,time_dehydr_dbe_wash2,dehydr_dbe_wash2_notes,clearing_notes
3,willmore,ahoag,,2019-02-28,2019-03-01 15:45:00,,2019-03-01 16:18:00,,2019-03-01 16:53:00,,2019-03-04 09:43:00,,2019-03-04 10:32:00,,2019-03-04 11:19:00,,2019-03-04 00:07:00,,2019-03-04 12:55:00,,2019-03-04 13:41:00,,None,,None,,None,,None,,None,,


In [71]:
os

<module 'os' from '/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/os.py'>

In [3]:
schema = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo') 

Please enter DataJoint username: ahoag
Please enter DataJoint password: ········
Connecting ahoag@localhost:3306


In [53]:
Experiment().fetch('title','description')

[array(['test'], dtype=object), array(['test'], dtype=object)]

In [54]:
attrs = ['title','description']

In [58]:
Experiment().fetch(*attrs,as_dict=True)

[{'title': 'test', 'description': 'test'}]

In [60]:
Experiment().fetch1(*attrs,as_dict=True)

TypeError: __call__() got an unexpected keyword argument 'as_dict'

In [183]:
'4'.zfill(3)

'004'

In [184]:
n=4

In [185]:
val = f'{n+1}'.zfill(3)
val

'005'